# Introduction

Being able to simulate fluids in computer graphics is essential for realistic rendering of water, smoke or any kind of gases. For this project, we decided to look into the classic paper of Jos Stam on stable fluids (1999) and to implement its algorithm on our own using Python 3 and Tensorflow. For now, we have a fully functional fluid simulator in 2D that is almost real-time. The original article can be found here: https://d2f99xq7vri1nk.cloudfront.net/legacy_app_files/pdf/ns.pdf

# The theory

As explained in the article, fluid dynamics are governed by the famous Navier-Stokes equations: 
    $$
    \left \{ \begin{aligned}
    & \nabla \cdot \mathbf{u} = 0  \\
    & \frac{\partial \mathbf{u}}{\partial t} = -(\mathbf{u} \cdot \nabla ) \mathbf{u} - \frac{1}{\rho} \nabla p + \nu \nabla ^2 \mathbf{u} + \mathbf{f}
    \end{aligned}\right.$$
    
where $\mathbf{u}$ is a vector field that represents the velocity and $p$ is the pressure scalar field. The first equation means that the fluid conserves the mass while the second one translates the fact the it conserves the momentum. The article uses the Helmholtz decomposition to assemble these two equations into one unique equation on the velocity field:
$$\frac{\partial \mathbf{u}}{\partial t} = \mathbf{P} \left( -(\mathbf{u} \cdot \nabla ) \mathbf{u} + \nu \nabla ^2 \mathbf{u} + \mathbf{f} \right)$$

where $- (\mathbf{u} \cdot \nabla ) \mathbf{u}$ is the advection term, $\nu \nabla^2 \mathbf{u}$ is the diffusion term and $\mathbf{f}$ is the force field applied to the fluid.

A grid is set to discretize space into cells and velocity is defined at the center of each cell. The equation is solved numerically at each iteration by computing four steps. We present the four steps and how we implemented them in 2D in the next section.

# Implementation

We decided to implement our project using Python 3 and Tensorflow 2 because of two reasons. For one thing, Python is an interpreted high-level programming language, so it allows quick coding without compilation. Also, Tensorflow 2, a library mainly used for machine learning purposes, offers a wide range of methods for manipulating multidimensional arrays, which is relevant for this project.

## Operators

Our first task was to implement the various operators that we will be using in the partial differential equations: gradient, divergence, laplacian and vector laplacian. These operators can be seen as convolutions between the main matrix (vector or scalar field) and a 3x3 or 5x5 kernel. Each of these kernels are combinations of first and second derivative kernels: 

$$\mathcal{K}_{\partial x} = \frac{1}{2}\begin{pmatrix} 0&0&0 \\ -1&0&1 \\ 0&0&0 \end{pmatrix} \qquad \mathcal{K}_{\partial y} = \frac{1}{2}\begin{pmatrix} 0&-1&0 \\ 0&0&0 \\ 0&1&0 \end{pmatrix} \qquad \mathcal{K}_{lap} = \begin{pmatrix} 0&1&0 \\ 1&-4&1 \\ 0&1&0 \end{pmatrix}$$ 



Let $\mathbf{U}(i,j)$ be a 2-dimensional vector field of size $m\times n$ where each coordinate $\textrm{U}_x(i,j)$ and $\textrm{U}_y(i,j)$ is a scalar field. Let $\textrm{S}(i,j)$ be a scalar field of size $m\times n$ too. Then we implemented the operators as following:

$\begin{aligned}
\bullet \quad &  \nabla \textrm{S} = \begin{pmatrix} \mathcal{K}_{\partial x} \ast S \\ \mathcal{K}_{\partial y} \ast S \end{pmatrix} \\
& \\
\bullet \quad &\nabla \cdot \mathbf{U} = \mathcal{K}_{\partial x} \ast \textrm{U}_x + \mathcal{K}_{\partial y} \ast \textrm{U}_y \\
& \\
\bullet \quad & \Delta \textrm{S} = \nabla \cdot \nabla \textrm{S} = \mathcal{K}_{\partial x} \ast (\mathcal{K}_{\partial x} \ast S) + \mathcal{K}_{\partial y} \ast (\mathcal{K}_{\partial y} \ast S) \\
& \\
\bullet \quad & \Delta \mathbf{U} = \begin{pmatrix} \mathcal{K}_{lap} \ast \textrm{U}_x \\ \mathcal{K}_{lap} \ast \textrm{U}_y \end{pmatrix} \\
\end{aligned}$

where $\ast$ is the convolution operation. Note that the laplacian and the vector laplacian are not computed with the same kernels: the first one is defined as a composition of the gradient and the divergence operators while the latter uses the 3x3 laplacian kernel defined earlier. This is closely related to the nature of the equations we will be solving with these two operators in the third and fourth step.

## First step: adding the force

The first step of each iteration consists of adding the external force field $\mathbf{f}$ to the velocity field of the previous step: 

$$\mathbf{w_1} = \mathbf{w_0} + \Delta t \cdot \mathbf{f}$$

The force field cannot be constant, otherwise the velocity field would explode over time. In our demo, we define the force as being zero in the whole grid except in a tube at the middle. In this vertical tube, the force equals $1-\mathbf{w_0}$. This will tend to keep the velocity near the value 1, which is relevant as we do not want particles in the grid to be able to cross more than one cell in a single time step.

## Second step: advection

As explained in the article, the particles are moved by the velocity field during each time step. Therefore, to obtain the velocity at a point $\mathbf{x}$ at the new time , we only need to backtrace the point $\mathbf{x}$ through the velocity field $\mathbf{w_1}$ over a time $\Delta t$. 

For each center of cell at position $\mathbf{x}$, we subtract the distance $\mathbf{w_1}\Delta t$ to it, giving us a new position on the grid: $\mathbf{x}' = \mathbf{x} - \mathbf{w_1}\Delta t$. Omitting border conditions for the moment, this position lies necessarily within four cell centers whose position are denoted by $\mathbf{x}_1$, $\mathbf{x}_2$, $\mathbf{x}_3$ and $\mathbf{x}_4$ (see figure). It can therefore by written as a convex combination of these four positions: $\mathbf{x}' = a\mathbf{x}_1 + b\mathbf{x}_2+c\mathbf{x}_3+d\mathbf{x}_4$, where $a+b+c+d =1$. The new velocity $\mathbf{w_2}(\mathbf x)$ at $\mathbf{x}$ that we are looking for can thus be written as: $\mathbf{w_2}(\mathbf x) = a\mathbf{w_1}(\mathbf x_1) + b\mathbf{w_1}(\mathbf x_2)+c\mathbf{w_1}(\mathbf x_3)+d\mathbf{w_1}(\mathbf x_4)$

In the case $\mathbf{x}'$ lies outside the grid, we clip its position values with the size of the grid to keep it within it. This solves the advection part of the equation.

## Third step: diffusion

The third step is about solving the diffusion component of the equation, it boils down to solving this linear system:

$$\left(\mathbf{I}-\nu \Delta t \nabla ^2\right)\mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})$$
with $\mathbf{w_3}$ being the unknown variable. 

Since we are not explicitly defining the matrix of the linear system (because we use convolution to compute the laplacian), we first tried traditional gradient descent to solve this equation. The error was not very high yet the compute time was relatively long, which did not allow real-time rendering of the fluid. We then tried using the conjugate gradient as an iterative method. This approach gave us faster results. 

```
conjugate_gradient
input: w2, nu, dt, error_max
output: w3
initialisation: 
x=w2 
r=w2 - x + dt*nu*lap(x) 
p=r
rsold= sum(r*r)
for i in range(1000):
    Ap = p - dt*nu*lap_vec(p)
    tmp = sum(p*Ap)
    alpha = rsold / tmp
    x=x + alpha * p
    r=r - alpha * Ap
    rsnew = sum(r*r)
    p=r + (rsnew / rsold) * p
    rsold=rsnew
    if sqrt(rsold) < error_max then
    break
end for
return x;
```

We initialize our unknown with the previous velocity field `w2` and update the various variables according to the method. We get pretty good approximations with around 100 iterations.

## Fourth step: projection

This step ensures that the new resulting field is divergence free. Basically, we need to solve $\nabla ^2 q = \nabla \cdot \mathbf{w}_3$ for $q$. The new field $\mathbf{w}_4 = \mathbf{w}_3 - \nabla q$ is then divergence free. Similarly, we use the conjugate gradient descent to solve this equation.

Note that the whole algorithm is indeed unconditionally stable for any time step since the velocity field computed at each step is a convex linear combination of the previous velocity field. Furthermore, the fixed boundary condition is valid because convolutions on border cells implicitly assume that all ghost cells outside our grid equals 0.

# Results and what's next

The current implementation is able to compute 5000 time steps in 8 minutes with a 80x100 pixel grid, which is around 10fps. The results look like this:
<img src="img/001.png" alt="Drawing" style="width: 200px;"/>
<img src="img/002.png" alt="Drawing" style="width: 200px;"/>
<img src="img/003.png" alt="Drawing" style="width: 200px;"/>
<img src="img/004.png" alt="Drawing" style="width: 200px;"/>

We still have a lot to do for this project. For instance, it could be interesting to test the implementation with various values of the viscosity $\mu$ to find the best one visually. We also have to translate the whole implementation into a 3-dimensional grid. For the rendering of the fluid in 3D, we are thinking of implementing a simple raytracing algorithm ourselves.
